In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import lightgbm
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['train_HK6lq50.csv', 'test_2nAIblo.csv']


In [2]:
train = pd.read_csv('../input/train_HK6lq50.csv') 
test = pd.read_csv('../input/test_2nAIblo.csv')

In [3]:
print(train.shape)
train.head()

(73147, 16)


,id,program_id,program_type,program_duration,test_id,test_type,difficulty_level,trainee_id,gender,education,city_tier,age,total_programs_enrolled,is_handicapped,trainee_engagement_rating,is_pass
0,9389_150,Y_1,Y,136,150,offline,intermediate,9389,M,Matriculation,3,24.0,5,N,1.0,0
1,16523_44,T_1,T,131,44,offline,easy,16523,F,High School Diploma,4,26.0,2,N,3.0,1
2,13987_178,Z_2,Z,120,178,online,easy,13987,M,Matriculation,1,40.0,1,N,2.0,1
3,13158_32,T_2,T,117,32,offline,easy,13158,F,Matriculation,3,NaN,4,N,1.0,1
4,10591_84,V_3,V,131,84,offline,intermediate,10591,F,High School Diploma,1,42.0,2,N,4.0,1


In [4]:
print(test.shape)
test.head()

(31349, 15)


,id,program_id,program_type,program_duration,test_id,test_type,difficulty_level,trainee_id,gender,education,city_tier,age,total_programs_enrolled,is_handicapped,trainee_engagement_rating
0,1626_45,T_1,T,131,45,offline,intermediate,1626,F,Matriculation,3,46.0,2,N,4.0
1,11020_130,Y_3,Y,135,130,online,easy,11020,M,Bachelors,3,NaN,4,N,4.0
2,12652_146,Y_2,Y,120,146,online,easy,12652,M,Matriculation,3,NaN,2,N,3.0
3,7038_72,V_4,V,122,72,offline,vary hard,7038,F,High School Diploma,1,NaN,2,N,2.0
4,888_71,V_4,V,122,71,offline,intermediate,888,F,Matriculation,3,NaN,2,N,2.0


In [5]:
train['source'] = 'train'
test['source'] = 'test'


In [6]:
train.head()

,id,program_id,program_type,program_duration,test_id,test_type,difficulty_level,trainee_id,gender,education,city_tier,age,total_programs_enrolled,is_handicapped,trainee_engagement_rating,is_pass,source
0,9389_150,Y_1,Y,136,150,offline,intermediate,9389,M,Matriculation,3,24.0,5,N,1.0,0,train
1,16523_44,T_1,T,131,44,offline,easy,16523,F,High School Diploma,4,26.0,2,N,3.0,1,train
2,13987_178,Z_2,Z,120,178,online,easy,13987,M,Matriculation,1,40.0,1,N,2.0,1,train
3,13158_32,T_2,T,117,32,offline,easy,13158,F,Matriculation,3,NaN,4,N,1.0,1,train
4,10591_84,V_3,V,131,84,offline,intermediate,10591,F,High School Diploma,1,42.0,2,N,4.0,1,train


In [7]:
data = pd.concat([train, test], ignore_index = True)
print(data.shape)

(104496, 17)


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  """Entry point for launching an IPython kernel.


In [8]:
data.head()

,age,city_tier,difficulty_level,education,gender,id,is_handicapped,is_pass,program_duration,program_id,program_type,source,test_id,test_type,total_programs_enrolled,trainee_engagement_rating,trainee_id
0,24.0,3,intermediate,Matriculation,M,9389_150,N,0.0,136,Y_1,Y,train,150,offline,5,1.0,9389
1,26.0,4,easy,High School Diploma,F,16523_44,N,1.0,131,T_1,T,train,44,offline,2,3.0,16523
2,40.0,1,easy,Matriculation,M,13987_178,N,1.0,120,Z_2,Z,train,178,online,1,2.0,13987
3,NaN,3,easy,Matriculation,F,13158_32,N,1.0,117,T_2,T,train,32,offline,4,1.0,13158
4,42.0,1,intermediate,High School Diploma,F,10591_84,N,1.0,131,V_3,V,train,84,offline,2,4.0,10591


In [9]:
data.columns

Index(['age', 'city_tier', 'difficulty_level', 'education', 'gender', 'id',
       'is_handicapped', 'is_pass', 'program_duration', 'program_id',
       'program_type', 'source', 'test_id', 'test_type',
       'total_programs_enrolled', 'trainee_engagement_rating', 'trainee_id'],
      dtype='object')

In [10]:

def  lable_encode_edulable_en (x):
    switcher = {
        "High School Diploma": 1,
        "Matriculation": 2,
        "Bachelors": 3,
        "Masters": 4,
    }
    return switcher.get(x, 0)

data["education"] = data["education"].apply(lambda x: lable_encode_edulable_en(x))

In [11]:
(data['education'].value_counts())
## consider the bachelors and masters in the same category
#data['education'] = data['education'].replace(4,3)
set(data['education'])

{0, 1, 2, 3, 4}

In [12]:
def lable_encode_diff(x):
    switcher = {
        "easy": 1,
        "intermediate": 2,
        "hard": 3,
        "vary hard": 4,
    }
    return switcher.get(x, 0)

data["difficulty_level"] = data["difficulty_level"].apply(lambda x: lable_encode_diff(x))

In [13]:
set(data['difficulty_level'])

{1, 2, 3, 4}

In [14]:
## add difficulty level and see the impact of it.
diffi = data["difficulty_level"]
edu = data["education"]

In [15]:
set(data['program_duration'])

{117, 120, 121, 122, 130, 131, 133, 134, 135, 136}

In [16]:
## try to add program_duration 

In [17]:
dummy_fields = ["city_tier","education","gender","is_handicapped","program_id","program_type","test_type"] # no program_duration test_id for it's feature importance
## adding difficulty level doesn't add value to LGBM
##try program_duration, doesn't add value
for item in dummy_fields:
    dummies = pd.get_dummies(data.loc[:, item], prefix=item) 
    data = pd.concat([data, dummies], axis = 1)
    data = data.drop(item, axis =1)

In [18]:
print(data.shape)
data.head()

(104496, 54)


,age,difficulty_level,id,is_pass,program_duration,source,test_id,total_programs_enrolled,trainee_engagement_rating,trainee_id,city_tier_1,city_tier_2,city_tier_3,city_tier_4,education_0,education_1,education_2,education_3,education_4,gender_F,gender_M,is_handicapped_N,is_handicapped_Y,program_id_S_1,program_id_S_2,program_id_T_1,program_id_T_2,program_id_T_3,program_id_T_4,program_id_U_1,program_id_U_2,program_id_V_1,program_id_V_2,program_id_V_3,program_id_V_4,program_id_X_1,program_id_X_2,program_id_X_3,program_id_Y_1,program_id_Y_2,program_id_Y_3,program_id_Y_4,program_id_Z_1,program_id_Z_2,program_id_Z_3,program_type_S,program_type_T,program_type_U,program_type_V,program_type_X,program_type_Y,program_type_Z,test_type_offline,test_type_online
0,24.0,2,9389_150,0.0,136,train,150,5,1.0,9389,0,0,1,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0
1,26.0,1,16523_44,1.0,131,train,44,2,3.0,16523,0,0,0,1,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0
2,40.0,1,13987_178,1.0,120,train,178,1,2.0,13987,1,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1
3,NaN,1,13158_32,1.0,117,train,32,4,1.0,13158,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0
4,42.0,2,10591_84,1.0,131,train,84,2,4.0,10591,1,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0


In [19]:
## add difficuty_level
##data['difficulty_level'] = diffi
## Adding redunduncy for difficulty_level affects the score badly, now make sure if there is any redunduncy remove it. 
## dont quit now, try to add education 
##data['education'] = edu ,it also dont works

In [22]:
def missing_flag(x):
    switcher = {
        True: 1,
    }
    return switcher.get(x, 0)

data["age_missing_flag"] = data.isnull()["age"].apply(lambda x:missing_flag(x))
data["rating_missing_flag"] = data.isnull()["trainee_engagement_rating"].apply(lambda x:missing_flag(x))

In [23]:
## fill the afe and trainee_engagement_rating
data['age'].fillna(data['age'].median(),inplace=True)  
## Let the LGBM handle the missing values 
print(data['trainee_engagement_rating'].mode())
data['trainee_engagement_rating'].fillna(data['trainee_engagement_rating'].mode(),inplace=True)

0    1.0
dtype: float64


In [24]:
print(data['trainee_engagement_rating'].median())
print(data['trainee_engagement_rating'].mode())


2.0
0    1.0
dtype: float64


In [25]:
data.columns

Index(['age', 'difficulty_level', 'id', 'is_pass', 'program_duration',
       'source', 'test_id', 'total_programs_enrolled',
       'trainee_engagement_rating', 'trainee_id', 'city_tier_1', 'city_tier_2',
       'city_tier_3', 'city_tier_4', 'education_0', 'education_1',
       'education_2', 'education_3', 'education_4', 'gender_F', 'gender_M',
       'is_handicapped_N', 'is_handicapped_Y', 'program_id_S_1',
       'program_id_S_2', 'program_id_T_1', 'program_id_T_2', 'program_id_T_3',
       'program_id_T_4', 'program_id_U_1', 'program_id_U_2', 'program_id_V_1',
       'program_id_V_2', 'program_id_V_3', 'program_id_V_4', 'program_id_X_1',
       'program_id_X_2', 'program_id_X_3', 'program_id_Y_1', 'program_id_Y_2',
       'program_id_Y_3', 'program_id_Y_4', 'program_id_Z_1', 'program_id_Z_2',
       'program_id_Z_3', 'program_type_S', 'program_type_T', 'program_type_U',
       'program_type_V', 'program_type_X', 'program_type_Y', 'program_type_Z',
       'test_type_offline', 'test

In [26]:
## separate train and test
train = data.loc[data['source']=='train']
test = data.loc[data['source']=='test']

In [27]:
train.drop('source',axis=1,inplace=True)
test.drop(['source','is_pass'],axis=1,inplace=True)

/opt/conda/lib/python3.6/site-packages/pandas/core/frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [28]:
y = train.is_pass.values
train.drop(['id', 'is_pass'], inplace=True, axis=1)

x, x_test, y, y_test = train_test_split(train, y, test_size=0.2, random_state=42, stratify=y)

/opt/conda/lib/python3.6/site-packages/pandas/core/frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [29]:
train_data = lightgbm.Dataset(x, label=y)
test_data = lightgbm.Dataset(x_test, label=y_test)

In [31]:
parameters = {
    'application': 'binary',
    'objective': 'binary',
    'feature_fraction':.8,
    'bagging_fraction':.8,
    'metric': 'auc',
    'is_unbalance': 'true',
    'boosting': 'gbdt',
    'verbose': 0
}

model = lightgbm.train(parameters,
                       train_data,
                       valid_sets=test_data,
                       num_boost_round=3500,  ## training longer than 2500 simply overfits it.
                       early_stopping_rounds=100)

[1]	valid_0's auc: 0.691087
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's auc: 0.697241
[3]	valid_0's auc: 0.698138
[4]	valid_0's auc: 0.69851
[5]	valid_0's auc: 0.706351
[6]	valid_0's auc: 0.709355
[7]	valid_0's auc: 0.710454
[8]	valid_0's auc: 0.710402
[9]	valid_0's auc: 0.713163
[10]	valid_0's auc: 0.713786
[11]	valid_0's auc: 0.715194
[12]	valid_0's auc: 0.716201
[13]	valid_0's auc: 0.71729
[14]	valid_0's auc: 0.717499
[15]	valid_0's auc: 0.718338
[16]	valid_0's auc: 0.720135
[17]	valid_0's auc: 0.721916
[18]	valid_0's auc: 0.72242
[19]	valid_0's auc: 0.722782
[20]	valid_0's auc: 0.724372
[21]	valid_0's auc: 0.724769
[22]	valid_0's auc: 0.725287
[23]	valid_0's auc: 0.725918
[24]	valid_0's auc: 0.726556
[25]	valid_0's auc: 0.726842
[26]	valid_0's auc: 0.727416
[27]	valid_0's auc: 0.727926
[28]	valid_0's auc: 0.728563
[29]	valid_0's auc: 0.729302
[30]	valid_0's auc: 0.729723
[31]	valid_0's auc: 0.730089
[32]	valid_0's auc: 0.730372
[33]	valid_0's auc: 0

[282]	valid_0's auc: 0.761759
[283]	valid_0's auc: 0.761889
[284]	valid_0's auc: 0.761878
[285]	valid_0's auc: 0.762043
[286]	valid_0's auc: 0.762218
[287]	valid_0's auc: 0.762276
[288]	valid_0's auc: 0.762437
[289]	valid_0's auc: 0.762569
[290]	valid_0's auc: 0.762528
[291]	valid_0's auc: 0.762678
[292]	valid_0's auc: 0.762687
[293]	valid_0's auc: 0.762674
[294]	valid_0's auc: 0.762645
[295]	valid_0's auc: 0.762802
[296]	valid_0's auc: 0.762806
[297]	valid_0's auc: 0.762844
[298]	valid_0's auc: 0.762821
[299]	valid_0's auc: 0.762964
[300]	valid_0's auc: 0.762983
[301]	valid_0's auc: 0.762982
[302]	valid_0's auc: 0.763015
[303]	valid_0's auc: 0.763111
[304]	valid_0's auc: 0.763178
[305]	valid_0's auc: 0.763189
[306]	valid_0's auc: 0.763273
[307]	valid_0's auc: 0.763384
[308]	valid_0's auc: 0.763369
[309]	valid_0's auc: 0.763566
[310]	valid_0's auc: 0.763615
[311]	valid_0's auc: 0.763764
[312]	valid_0's auc: 0.763793
[313]	valid_0's auc: 0.76386
[314]	valid_0's auc: 0.763842
[315]	valid

[558]	valid_0's auc: 0.772973
[559]	valid_0's auc: 0.772885
[560]	valid_0's auc: 0.772917
[561]	valid_0's auc: 0.772914
[562]	valid_0's auc: 0.772976
[563]	valid_0's auc: 0.772945
[564]	valid_0's auc: 0.772943
[565]	valid_0's auc: 0.772994
[566]	valid_0's auc: 0.773027
[567]	valid_0's auc: 0.773073
[568]	valid_0's auc: 0.773106
[569]	valid_0's auc: 0.773137
[570]	valid_0's auc: 0.773153
[571]	valid_0's auc: 0.773183
[572]	valid_0's auc: 0.773176
[573]	valid_0's auc: 0.773191
[574]	valid_0's auc: 0.773214
[575]	valid_0's auc: 0.77318
[576]	valid_0's auc: 0.773226
[577]	valid_0's auc: 0.77329
[578]	valid_0's auc: 0.773325
[579]	valid_0's auc: 0.773344
[580]	valid_0's auc: 0.773469
[581]	valid_0's auc: 0.773592
[582]	valid_0's auc: 0.773585
[583]	valid_0's auc: 0.773658
[584]	valid_0's auc: 0.773677
[585]	valid_0's auc: 0.773757
[586]	valid_0's auc: 0.773831
[587]	valid_0's auc: 0.773856
[588]	valid_0's auc: 0.773913
[589]	valid_0's auc: 0.773912
[590]	valid_0's auc: 0.773904
[591]	valid_

[836]	valid_0's auc: 0.779485
[837]	valid_0's auc: 0.779472
[838]	valid_0's auc: 0.779583
[839]	valid_0's auc: 0.779532
[840]	valid_0's auc: 0.77954
[841]	valid_0's auc: 0.779479
[842]	valid_0's auc: 0.779534
[843]	valid_0's auc: 0.779506
[844]	valid_0's auc: 0.779575
[845]	valid_0's auc: 0.779545
[846]	valid_0's auc: 0.779505
[847]	valid_0's auc: 0.779475
[848]	valid_0's auc: 0.779448
[849]	valid_0's auc: 0.779453
[850]	valid_0's auc: 0.779421
[851]	valid_0's auc: 0.77939
[852]	valid_0's auc: 0.779395
[853]	valid_0's auc: 0.779414
[854]	valid_0's auc: 0.779414
[855]	valid_0's auc: 0.779436
[856]	valid_0's auc: 0.779436
[857]	valid_0's auc: 0.779493
[858]	valid_0's auc: 0.779524
[859]	valid_0's auc: 0.779529
[860]	valid_0's auc: 0.779588
[861]	valid_0's auc: 0.779612
[862]	valid_0's auc: 0.77961
[863]	valid_0's auc: 0.779588
[864]	valid_0's auc: 0.779612
[865]	valid_0's auc: 0.779668
[866]	valid_0's auc: 0.779665
[867]	valid_0's auc: 0.779668
[868]	valid_0's auc: 0.779702
[869]	valid_0

[1110]	valid_0's auc: 0.782359
[1111]	valid_0's auc: 0.782388
[1112]	valid_0's auc: 0.782406
[1113]	valid_0's auc: 0.782429
[1114]	valid_0's auc: 0.782465
[1115]	valid_0's auc: 0.782437
[1116]	valid_0's auc: 0.782423
[1117]	valid_0's auc: 0.782366
[1118]	valid_0's auc: 0.782397
[1119]	valid_0's auc: 0.782364
[1120]	valid_0's auc: 0.7824
[1121]	valid_0's auc: 0.782408
[1122]	valid_0's auc: 0.782427
[1123]	valid_0's auc: 0.782452
[1124]	valid_0's auc: 0.782523
[1125]	valid_0's auc: 0.782564
[1126]	valid_0's auc: 0.782523
[1127]	valid_0's auc: 0.782597
[1128]	valid_0's auc: 0.782582
[1129]	valid_0's auc: 0.782511
[1130]	valid_0's auc: 0.782483
[1131]	valid_0's auc: 0.782491
[1132]	valid_0's auc: 0.782505
[1133]	valid_0's auc: 0.782539
[1134]	valid_0's auc: 0.782521
[1135]	valid_0's auc: 0.782589
[1136]	valid_0's auc: 0.782602
[1137]	valid_0's auc: 0.782604
[1138]	valid_0's auc: 0.782602
[1139]	valid_0's auc: 0.782598
[1140]	valid_0's auc: 0.782643
[1141]	valid_0's auc: 0.782624
[1142]	val

...

[1647]	valid_0's auc: 0.786527
[1648]	valid_0's auc: 0.786568
[1649]	valid_0's auc: 0.786583
[1650]	valid_0's auc: 0.786565
[1651]	valid_0's auc: 0.78651
[1652]	valid_0's auc: 0.786537
[1653]	valid_0's auc: 0.786599
[1654]	valid_0's auc: 0.786587
[1655]	valid_0's auc: 0.786594
[1656]	valid_0's auc: 0.786598
[1657]	valid_0's auc: 0.786639
[1658]	valid_0's auc: 0.786678
[1659]	valid_0's auc: 0.786647
[1660]	valid_0's auc: 0.786668
[1661]	valid_0's auc: 0.786679
[1662]	valid_0's auc: 0.786681
[1663]	valid_0's auc: 0.786691
[1664]	valid_0's auc: 0.786698
[1665]	valid_0's auc: 0.786707
[1666]	valid_0's auc: 0.786704
[1667]	valid_0's auc: 0.786684
[1668]	valid_0's auc: 0.786691
[1669]	valid_0's auc: 0.786717
[1670]	valid_0's auc: 0.786701
[1671]	valid_0's auc: 0.78668
[1672]	valid_0's auc: 0.786655
[1673]	valid_0's auc: 0.786635
[1674]	valid_0's auc: 0.786623
[1675]	valid_0's auc: 0.786618
[1676]	valid_0's auc: 0.78665
[1677]	valid_0's auc: 0.78666
[1678]	valid_0's auc: 0.786727
[1679]	valid

[1922]	valid_0's auc: 0.787194
[1923]	valid_0's auc: 0.787227
[1924]	valid_0's auc: 0.787228
[1925]	valid_0's auc: 0.787236
[1926]	valid_0's auc: 0.787221
[1927]	valid_0's auc: 0.787236
[1928]	valid_0's auc: 0.787237
[1929]	valid_0's auc: 0.787247
[1930]	valid_0's auc: 0.787223
[1931]	valid_0's auc: 0.787236
[1932]	valid_0's auc: 0.787221
[1933]	valid_0's auc: 0.78724
[1934]	valid_0's auc: 0.787242
[1935]	valid_0's auc: 0.787242
[1936]	valid_0's auc: 0.787247
[1937]	valid_0's auc: 0.787259
[1938]	valid_0's auc: 0.787244
[1939]	valid_0's auc: 0.787264
[1940]	valid_0's auc: 0.787266
[1941]	valid_0's auc: 0.787254
[1942]	valid_0's auc: 0.787229
[1943]	valid_0's auc: 0.787237
[1944]	valid_0's auc: 0.787234
[1945]	valid_0's auc: 0.787215
[1946]	valid_0's auc: 0.78721
[1947]	valid_0's auc: 0.787209
[1948]	valid_0's auc: 0.787213
[1949]	valid_0's auc: 0.787202
[1950]	valid_0's auc: 0.787184
[1951]	valid_0's auc: 0.787204
[1952]	valid_0's auc: 0.787193
[1953]	valid_0's auc: 0.787204
[1954]	val

[2197]	valid_0's auc: 0.787434
[2198]	valid_0's auc: 0.787454
[2199]	valid_0's auc: 0.787437
[2200]	valid_0's auc: 0.787459
[2201]	valid_0's auc: 0.787453
[2202]	valid_0's auc: 0.78744
[2203]	valid_0's auc: 0.787439
[2204]	valid_0's auc: 0.787435
[2205]	valid_0's auc: 0.787439
[2206]	valid_0's auc: 0.787476
[2207]	valid_0's auc: 0.787482
[2208]	valid_0's auc: 0.787468
[2209]	valid_0's auc: 0.787452
[2210]	valid_0's auc: 0.787439
[2211]	valid_0's auc: 0.787484
[2212]	valid_0's auc: 0.787449
[2213]	valid_0's auc: 0.787465
[2214]	valid_0's auc: 0.787435
[2215]	valid_0's auc: 0.787469
[2216]	valid_0's auc: 0.787445
[2217]	valid_0's auc: 0.787467
[2218]	valid_0's auc: 0.787469
[2219]	valid_0's auc: 0.78746
[2220]	valid_0's auc: 0.787482
[2221]	valid_0's auc: 0.787449
[2222]	valid_0's auc: 0.787425
[2223]	valid_0's auc: 0.787434
[2224]	valid_0's auc: 0.787431
[2225]	valid_0's auc: 0.787475
[2226]	valid_0's auc: 0.787556
[2227]	valid_0's auc: 0.787567
[2228]	valid_0's auc: 0.78757
[2229]	vali

[2473]	valid_0's auc: 0.788035
[2474]	valid_0's auc: 0.788018
[2475]	valid_0's auc: 0.788033
[2476]	valid_0's auc: 0.788056
[2477]	valid_0's auc: 0.788055
[2478]	valid_0's auc: 0.788075
[2479]	valid_0's auc: 0.788073
[2480]	valid_0's auc: 0.788101
[2481]	valid_0's auc: 0.788093
[2482]	valid_0's auc: 0.788086
[2483]	valid_0's auc: 0.788108
[2484]	valid_0's auc: 0.78812
[2485]	valid_0's auc: 0.788097
[2486]	valid_0's auc: 0.788097
[2487]	valid_0's auc: 0.788077
[2488]	valid_0's auc: 0.788096
[2489]	valid_0's auc: 0.788076
[2490]	valid_0's auc: 0.788086
[2491]	valid_0's auc: 0.78809
[2492]	valid_0's auc: 0.788101
[2493]	valid_0's auc: 0.788119
[2494]	valid_0's auc: 0.788113
[2495]	valid_0's auc: 0.78811
[2496]	valid_0's auc: 0.788129
[2497]	valid_0's auc: 0.788103
[2498]	valid_0's auc: 0.788105
[2499]	valid_0's auc: 0.788105
[2500]	valid_0's auc: 0.78812
[2501]	valid_0's auc: 0.788127
[2502]	valid_0's auc: 0.788132
[2503]	valid_0's auc: 0.788121
[2504]	valid_0's auc: 0.788116
[2505]	valid

[2740]	valid_0's auc: 0.788775
[2741]	valid_0's auc: 0.788796
[2742]	valid_0's auc: 0.788789
[2743]	valid_0's auc: 0.788785
[2744]	valid_0's auc: 0.788791
[2745]	valid_0's auc: 0.788791
[2746]	valid_0's auc: 0.788814
[2747]	valid_0's auc: 0.788811
[2748]	valid_0's auc: 0.788812
[2749]	valid_0's auc: 0.788788
[2750]	valid_0's auc: 0.788795
[2751]	valid_0's auc: 0.788841
[2752]	valid_0's auc: 0.78884
[2753]	valid_0's auc: 0.788822
[2754]	valid_0's auc: 0.788825
[2755]	valid_0's auc: 0.788818
[2756]	valid_0's auc: 0.78883
[2757]	valid_0's auc: 0.788844
[2758]	valid_0's auc: 0.788816
[2759]	valid_0's auc: 0.788828
[2760]	valid_0's auc: 0.788855
[2761]	valid_0's auc: 0.788859
[2762]	valid_0's auc: 0.788872
[2763]	valid_0's auc: 0.788882
[2764]	valid_0's auc: 0.788872
[2765]	valid_0's auc: 0.788854
[2766]	valid_0's auc: 0.78883
[2767]	valid_0's auc: 0.788812
[2768]	valid_0's auc: 0.78882
[2769]	valid_0's auc: 0.788855
[2770]	valid_0's auc: 0.788888
[2771]	valid_0's auc: 0.78891
[2772]	valid_

In [32]:
## Best train auc score is .788 
ids = test['id'].values
test.drop('id', inplace=True, axis=1)

/opt/conda/lib/python3.6/site-packages/pandas/core/frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [33]:
x = test.values
y = model.predict(x)

y = list(y)

In [34]:
output = pd.DataFrame({'id': ids, 'is_pass': y})
output.to_csv('lgb.csv', index=False)
output.head()

,id,is_pass
0,1626_45,0.245345
1,11020_130,0.966852
2,12652_146,0.367842
3,7038_72,0.058712
4,888_71,0.121246
